# Topic Modelling
Data Science Workflow:
1. Start with a question 
2. Get & Clean the data   
3. Perform EDA
4. Apply Techniques
    - **Input:** clean data, plus we have verified that the data makes sense
    - **NLP Techniques:** advanced analysis techniques, in this case, they are specifically designed for text data
    - **Output:** additional insights about our data to help us answer our question, "how is Ali Wong different?"
5. Share Insights

## Topic Modeling
- **Input:** A document-term matrix. Each topic will consist of a set of words where order does not matter, so we are going to start with the bag of words format.
- **genism:** genism is a Python toolkit built by Radim Rehurek specifically for topic modeling. We are going to a popular topic modeling technique called Latent Dirichlet Allocation (LDA). We are also going to use **nltk** for some parts-of-speech tagging.
- **Output:** Our goal is to find themes across various comedy routines, and see which comedians tend to talk about which themes.

### Latent Dirichlet Allocation
Latent -- > hidden <br>
Dirichlet --> types of probability distribution

### How LDA works?
- **Goal:** - You want LDA to learn the `topic mix` in each document, and the `word mix` in each topic
- **Input:** - Document-Term Matrix, number of topics, number of iterations
- **Genism** will go through the process of finding the best word distribution for each topic and best topic distribution for each document.
- **Output:** *The top words in each topic*. It is your job as a human to interpret this and see if the results makes sense. If not, try altering the parameters - terms in the document-term matrix, number of topics, number of iterations, etc, *Stop when the topics make sense*.
- This is a probabilistic approach to topic modelling. There are also matriz factorization techniques for topic modeling such as **Latent Semantic Indexing (LSI)** and **Non-Negative Matrix Factorization (NMF)**


## 1.2 Topic Modelling -  Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import dill

data = pd.read_pickle('dtm_stop.pkl')
data

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aaah,aah,abc,ability,abject,able,...,zealand,zee,zen,zeppelin,zero,zillion,zombie,zoning,zoo,éclair
ali,0,0,0,0,0,0,1,0,0,2,...,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,1,2,1,0,0
bo,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
dave,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,1,...,0,2,1,0,1,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,0,0,3,0,0,0,1,...,0,0,0,0,2,0,0,0,0,0


In [2]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,bill,bo,dave,hasan,jim,joe,john,louis,mike,ricky
aaaaah,0,0,1,0,0,0,0,0,0,0,0,0
aaaaahhhhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaaaauuugghhhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaaahhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaah,0,0,0,0,1,0,0,0,0,0,0,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, 
# from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [6]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = dill.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
# id2word

In [7]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.007*"cause" + 0.006*"went" + 0.006*"really" + 0.005*"fucking" + 0.005*"day" + 0.005*"ha" + 0.005*"oh" + 0.005*"good" + 0.005*"come" + 0.005*"going"'),
 (1,
  '0.009*"fucking" + 0.007*"shit" + 0.007*"yeah" + 0.007*"fuck" + 0.006*"want" + 0.005*"come" + 0.005*"going" + 0.005*"didnt" + 0.005*"theyre" + 0.005*"day"')]

In [8]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.008*"fucking" + 0.007*"shit" + 0.007*"woman" + 0.006*"cause" + 0.006*"day" + 0.006*"went" + 0.006*"fuck" + 0.006*"come" + 0.006*"life" + 0.006*"theyre"'),
 (1,
  '0.010*"fucking" + 0.008*"yeah" + 0.007*"fuck" + 0.007*"shit" + 0.007*"want" + 0.005*"didnt" + 0.005*"going" + 0.005*"come" + 0.005*"kid" + 0.005*"theyre"'),
 (2,
  '0.006*"cause" + 0.005*"really" + 0.005*"mean" + 0.005*"way" + 0.005*"friend" + 0.004*"oh" + 0.004*"point" + 0.004*"jenny" + 0.004*"day" + 0.004*"yeah"')]

In [9]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.009*"love" + 0.008*"bo" + 0.008*"stuff" + 0.007*"repeat" + 0.006*"want" + 0.006*"yeah" + 0.005*"eye" + 0.005*"slut" + 0.005*"fucking" + 0.005*"um"'),
 (1,
  '0.011*"fucking" + 0.009*"went" + 0.009*"day" + 0.007*"joke" + 0.007*"gun" + 0.007*"fuck" + 0.006*"come" + 0.006*"going" + 0.006*"oh" + 0.006*"good"'),
 (2,
  '0.011*"dad" + 0.007*"going" + 0.007*"want" + 0.007*"love" + 0.006*"shes" + 0.006*"hasan" + 0.005*"look" + 0.005*"life" + 0.005*"mom" + 0.004*"oh"'),
 (3,
  '0.008*"fucking" + 0.008*"shit" + 0.007*"yeah" + 0.006*"fuck" + 0.006*"theyre" + 0.006*"didnt" + 0.006*"cause" + 0.005*"kid" + 0.005*"come" + 0.005*"woman"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## 1.3 Topic Modeling - Attempt # 2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [10]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [11]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
ali,lady and gentleman please welcome to the stage...
anthony,thank you thank you thank you san francisco th...
bill,all right thank you thank you very much thank ...
bo,bo what old macdonald had a farm e i e i o and...
dave,this is dave he tell dirty joke for a living t...
hasan,whats up davis whats up im home i had to bring...
jim,lady and gentleman please welcome to the stage...
joe,lady and gentleman welcome joe rogan what the ...
john,all right petunia wish me luck out there you w...
louis,intro fade the music out let roll hold there l...


In [12]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yrobi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [13]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yrobi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,lady please stage ali hi thank hello na shit c...
anthony,thank thank people i em i gon tape francisco c...
bill,thank thank pleasure georgia area oasis i june...
bo,macdonald farm e i o farm pig e i i snort macd...
dave,joke living stare work profound train thought ...
hasan,whats davis whats home i netflix la york i son...
jim,lady please stage mr jim jefferies thank thats...
joe,lady joe fuck thanks phone fuckface i phone mo...
john,petunia thats hello hello chicago thank crowd ...
louis,intro music let roll light thank i i place pla...


In [15]:

# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
# Convert frozenset to a list
stop_words_list = list(stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words_list)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaaaahhhhhhh,aaaaauuugghhhhhh,aaah,aah,abc,ability,abortion,abuse,acc,accent,...,yulin,yummy,ze,zealand,zee,zeppelin,zillion,zombie,zoo,éclair
ali,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
anthony,0,0,0,0,0,0,2,0,0,1,...,0,0,0,8,0,0,0,0,0,0
bill,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,2,0,0
bo,1,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [17]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.015*"thing" + 0.011*"man" + 0.010*"shit" + 0.010*"wa" + 0.010*"day" + 0.010*"fuck" + 0.009*"woman" + 0.009*"guy" + 0.008*"gon" + 0.008*"kid"'),
 (1,
  '0.012*"thing" + 0.010*"day" + 0.010*"life" + 0.010*"wa" + 0.008*"kid" + 0.008*"dad" + 0.008*"way" + 0.008*"cause" + 0.007*"guy" + 0.007*"woman"')]

In [18]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.016*"thing" + 0.013*"day" + 0.011*"shit" + 0.011*"woman" + 0.009*"fuck" + 0.009*"gon" + 0.009*"kid" + 0.009*"man" + 0.009*"wa" + 0.009*"guy"'),
 (1,
  '0.012*"wa" + 0.011*"thing" + 0.009*"day" + 0.008*"way" + 0.008*"dad" + 0.007*"guy" + 0.007*"man" + 0.007*"life" + 0.006*"night" + 0.006*"year"'),
 (2,
  '0.017*"life" + 0.016*"thing" + 0.010*"tit" + 0.009*"kid" + 0.009*"guy" + 0.008*"woman" + 0.007*"course" + 0.007*"cause" + 0.006*"way" + 0.005*"gon"')]

In [19]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.019*"thing" + 0.012*"woman" + 0.012*"wa" + 0.012*"day" + 0.011*"guy" + 0.011*"shit" + 0.010*"life" + 0.010*"man" + 0.009*"kid" + 0.008*"year"'),
 (1,
  '0.015*"fuck" + 0.012*"thing" + 0.011*"man" + 0.011*"kid" + 0.010*"house" + 0.010*"shit" + 0.010*"gon" + 0.009*"theyre" + 0.008*"life" + 0.008*"dude"'),
 (2,
  '0.011*"day" + 0.010*"dad" + 0.009*"thing" + 0.008*"life" + 0.008*"joke" + 0.007*"shit" + 0.007*"lot" + 0.007*"wa" + 0.007*"year" + 0.007*"kid"'),
 (3,
  '0.012*"wa" + 0.010*"cause" + 0.010*"thing" + 0.009*"way" + 0.009*"guy" + 0.008*"day" + 0.008*"friend" + 0.008*"point" + 0.007*"kind" + 0.007*"night"')]

## 1.4 Topic Modeling - Attempt # 3 (Nouns and Adjectives)

In [20]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [21]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,lady gentleman please stage ali wong hi welcom...
anthony,thank san francisco thank good people surprise...
bill,right thank thank pleasure greater atlanta geo...
bo,old macdonald farm e i i o farm pig e i i snor...
dave,dave dirty joke living stare most hard work pr...
hasan,whats davis whats im home i netflix special la...
jim,lady gentleman please stage mr jim jefferies t...
joe,lady gentleman welcome joe fuck san francisco ...
john,right petunia august thats good right hello he...
louis,intro music let roll light light thank much i ...


In [22]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words_list, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aaah,aah,abc,ability,abject,able,...,yyou,ze,zealand,zee,zeppelin,zero,zillion,zombie,zoo,éclair
ali,0,0,0,0,0,0,1,0,0,2,...,1,0,0,0,0,0,0,1,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,8,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,1,2,0,0
bo,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
dave,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,1,...,0,0,0,2,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,0,0,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [24]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"fucking" + 0.006*"gun" + 0.004*"wife" + 0.004*"son" + 0.003*"let" + 0.003*"dog" + 0.003*"cunt" + 0.003*"hell" + 0.003*"american" + 0.003*"gay"'),
 (1,
  '0.004*"parent" + 0.003*"fucking" + 0.003*"jenny" + 0.003*"ok" + 0.003*"comedy" + 0.003*"bo" + 0.003*"tit" + 0.003*"sure" + 0.003*"hasan" + 0.003*"door"')]

In [25]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.009*"fucking" + 0.007*"gun" + 0.004*"dog" + 0.004*"tit" + 0.003*"cunt" + 0.003*"nut" + 0.003*"hell" + 0.003*"girlfriend" + 0.003*"class" + 0.003*"let"'),
 (1,
  '0.005*"ahah" + 0.005*"mad" + 0.005*"anthony" + 0.005*"husband" + 0.005*"nigga" + 0.004*"grandma" + 0.004*"shark" + 0.004*"asian" + 0.004*"gay" + 0.003*"ok"'),
 (2,
  '0.006*"parent" + 0.005*"jenny" + 0.004*"bo" + 0.004*"hasan" + 0.004*"comedy" + 0.003*"repeat" + 0.003*"clinton" + 0.003*"york" + 0.003*"contact" + 0.003*"cow"')]

In [26]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"hasan" + 0.005*"parent" + 0.005*"gun" + 0.005*"fucking" + 0.005*"dream" + 0.004*"anthony" + 0.004*"comedy" + 0.004*"mad" + 0.004*"birthday" + 0.004*"shark"'),
 (1,
  '0.007*"jenny" + 0.007*"bo" + 0.006*"tit" + 0.005*"repeat" + 0.004*"contact" + 0.004*"song" + 0.004*"andy" + 0.004*"comedy" + 0.004*"dog" + 0.004*"parent"'),
 (2,
  '0.005*"husband" + 0.005*"nut" + 0.004*"chimp" + 0.004*"doctor" + 0.004*"jenner" + 0.004*"ok" + 0.004*"fucking" + 0.004*"pregnant" + 0.004*"rape" + 0.004*"twitter"'),
 (3,
  '0.008*"fucking" + 0.006*"gun" + 0.005*"wife" + 0.004*"ahah" + 0.004*"son" + 0.004*"american" + 0.004*"clinton" + 0.003*"nigga" + 0.003*"door" + 0.003*"president"')]

## 1.5 Topic Modeling - final 

In [27]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.013*"jenny" + 0.005*"accident" + 0.005*"sure" + 0.005*"marriage" + 0.005*"scrambler" + 0.004*"argument" + 0.004*"andy" + 0.004*"club" + 0.004*"idea" + 0.004*"morning"'),
 (1,
  '0.007*"fucking" + 0.004*"dog" + 0.004*"wife" + 0.004*"tit" + 0.004*"parent" + 0.003*"nut" + 0.003*"president" + 0.003*"clinton" + 0.003*"let" + 0.003*"hell"'),
 (2,
  '0.010*"gun" + 0.007*"fucking" + 0.006*"cunt" + 0.006*"husband" + 0.005*"girlfriend" + 0.005*"business" + 0.005*"ok" + 0.005*"pregnant" + 0.004*"asian" + 0.004*"ticket"'),
 (3,
  '0.005*"bo" + 0.005*"comedy" + 0.005*"hasan" + 0.004*"mad" + 0.004*"parent" + 0.004*"ahah" + 0.004*"repeat" + 0.004*"dream" + 0.004*"gay" + 0.004*"anthony"')]

In [30]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(2, 'ali'),
 (3, 'anthony'),
 (1, 'bill'),
 (3, 'bo'),
 (3, 'dave'),
 (3, 'hasan'),
 (2, 'jim'),
 (1, 'joe'),
 (1, 'john'),
 (1, 'louis'),
 (0, 'mike'),
 (1, 'ricky')]

For a first pass of LDA, these kind of make sense.
* Topic 0: mom, parents [Anthony, Hasan, Louis, Ricky]
* Topic 1: husband, wife [Ali, John, Mike]
* Topic 2: guns [Bill, Bo, Jim]
* Topic 3: profanity [Dave, Joe]